In [73]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math
from ipywidgets import widgets, interact
from IPython.display import display
import pandas as pd
import seaborn as sns

In [74]:
#list of constants here
theta = 0.244 #in radians (0 - 0.7853)
rho =  1.15*10**9 #in ng/mL
h =  0.06 #in cm
r = 0.015 #radius microneedle cm
cs = 1.34*10**6 #in ng/mL
beta = 0.185 #unitless 0-1
kd = 0.007 #cm/s
gamma = 1.16 #unitless
nMN = 100 #number of needles in patch
Vs = 0.049 #mL volume of skin compartment
Vn = (1/3)*math.pi*r**2*h #volume of 1 microneedle in mL - cm3
Bbp = 0.7 #size of side base plate 
hbp = 0.26 #height of base pair
Vbp =  Bbp*hbp #volume baseplase in mL or cm3
diff = 0.3  #drug diffusivity cm2/s
dose = 3*10**5
doseMN = beta*Vn*rho
doseBP = beta*Vbp*rho
hderm = 0.1 #thickness of dermis in cm
Clb = 470.8 #systemic clearance of drug
Vb = 5.22*10**6 #volume of distribution blood in mL
dose = 3*10**5 #total dose ng

In [75]:
def eqn_rate_drug_release_MN(DRMN_time, theta, rho, h, cs, DCS_time, beta, kd):
    A = beta * rho * np.pi * (np.tan(theta))**2  # Added beta
    deltaZMN = (3 * DRMN_time / A) ** (1/3)
    deltaZMN = np.minimum(deltaZMN, h)  # Cap at needle height
    
    area_DMN = (np.pi * np.tan(theta) / np.cos(theta) * (h**2 - deltaZMN**2) 
                + np.pi * (deltaZMN * np.tan(theta))**2)
    area_DMN = np.maximum(area_DMN, 0)
    
    dDRMNdt = beta * kd * area_DMN * (cs - ((1 - beta) / beta) * DCS_time)
    dDRMNdt = np.maximum(dDRMNdt, 0)
    return dDRMNdt

In [76]:
def eqn_rate_drug_release_BP(DRBP_time, rho, Bbp, DCS_time, beta, kd, hbp, cs):
    deltaZBP = DRBP_time / (beta * rho * Bbp**2)  # Added beta
    deltaZBP = float(deltaZBP)
    # Constant contact area until fully dissolved
    if deltaZBP >= hbp:
        area_DBP = 0.0
    else:
        area_DBP = Bbp**2
    
    dDRBPdt = beta * kd * area_DBP * (cs - ((1 - beta) / beta) * DCS_time)
    dDRBPdt = np.maximum(dDRBPdt, 0)
    return dDRBPdt

In [77]:
def eqn_rate_flux_drugBP_2skin(nMN, diff, hderm, hbp, DBP, Vbp, DCS_time, r,dDRBPdt):
    CBP = DBP / Vbp  # Actual concentration in dissolved pool
    dFDBPS = (2 * diff * nMN * np.pi * r**2) / (hderm + hbp) * (CBP - DCS_time)
    
    dDBP_dt = dDRBPdt - dFDBPS
    return [dFDBPS, dDBP_dt]

In [78]:
#function for rate of change of drug concentration in skin
def eqn_rate_drug_concentration_skin(nMN,Vs,gamma, dDRMNdt,dFDBPS, DCS_time):
    # look here
    temp1 = nMN/Vs*(dDRMNdt)
    temp2 = np.array(dFDBPS)/Vs
    temp3 = gamma*DCS_time
    dDCSdt = temp1 + temp2 - temp3
    return dDCSdt

In [79]:
def eqn_change_DCP(DCS_time, Vs, Vb, gamma, Clb, DCP_time):
    dDCPdt = (gamma * Vs * DCS_time - Clb * DCP_time) / Vb
    return dDCPdt

In [80]:
controlMN = beta * rho * nMN * Vn   # Total drug in all microneedles
controlBP = beta * rho * Vbp         # Total drug in baseplate

In [81]:
def system_of_odes(t, y):
    DRMN_time, DRBP_solid, DBP_time, DCS_time, DCP_time = y
    # DRMN_time:  cumulative drug released from MN solid (ng)
    # DRBP_solid: cumulative drug released from BP solid (ng)
    # DBP:        dissolved drug currently in BP pool (ng)
    # DCS_time:   drug concentration in skin (ng/mL)
    # DCP_time:   drug concentration in plasma (ng/mL)
    
    # Microneedle dissolution (directly into skin)
    dDRMNdt = eqn_rate_drug_release_MN(DRMN_time, theta, rho, h, cs, DCS_time, beta, kd)
    if DRMN_time >= controlMN:
        dDRMNdt = 0
    
    # Baseplate solid dissolution (into BP dissolved pool)
    dDRBPdt = eqn_rate_drug_release_BP(DRBP_solid, rho, Bbp, DCS_time, beta, kd, hbp, cs)
    if DRBP_solid >= controlBP:
        dDRBPdt = 0
    
    # Flux from BP dissolved pool to skin (uses DBP, not DRBP_solid)
    [dFDBPS, dDBP_dt] = eqn_rate_flux_drugBP_2skin(nMN, diff, hderm, hbp, DBP_time, Vbp, DCS_time, r,dDRBPdt)

    
    if DRBP_solid >= controlBP and DBP_time <= 0:
        dFDBPS = 0
    
    # Mass balance for BP dissolved pool: +dissolution, -flux to skin
    dDBPdt = dDRBPdt - dFDBPS
    
    # Skin concentration
    dDCSdt = eqn_rate_drug_concentration_skin(nMN, Vs, gamma, dDRMNdt, dFDBPS, DCS_time)
    
    # Plasma concentration 
    dDCPdt = eqn_change_DCP(DCS_time, Vs, Vb, gamma, Clb, DCP_time)
    
    return [dDRMNdt, dDRBPdt, dDBPdt, dDCSdt, dDCPdt]

In [ ]:
y0 = [0.0, 0.0, 0.0, 0.0, 0.0]  # [DRMN, DRBP_solid, DBP, DCS, DCP]
t_span = (0.0, 120000.0)
t_eval = np.linspace(0.0, 120000.0, 100)
sol = solve_ivp(system_of_odes, t_span, y0)

In [ ]:
# ============ INDIVIDUAL PLOTS ============
fig, axes = plt.subplots(3, 2, figsize=(12, 12))

# Plot 1: Cumulative drug released from microneedles
axes[0, 0].plot(sol.t, sol.y[0]/1e3, label='Released Drug(t) from MN')
axes[0, 0].set_xlabel('Time (sec)')
axes[0, 0].set_ylabel('Cumulative Release (µg)')
axes[0, 0].set_title('Drug Released from Microneedles (DRMN)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Cumulative drug released from baseplate solid
axes[0, 1].plot(sol.t, sol.y[1]/1e3, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Time (sec)')
axes[0, 1].set_ylabel('Cumulative Release (µg)')
axes[0, 1].set_title('Drug Released from Baseplate Solid (DRBP)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Dissolved drug pool in baseplate
axes[1, 0].plot(sol.t, sol.y[2] / 1e3, 'orange', linewidth=2)
axes[1, 0].set_xlabel('Time (sec)')
axes[1, 0].set_ylabel('Drug Amount (µg)')

axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Drug concentration in skin
axes[1, 1].plot(sol.t, sol.y[3], 'g-', linewidth=2)
axes[1, 1].set_xlabel('Time (sec)')
axes[1, 1].set_ylabel('Concentration (ng/mL)')
axes[1, 1].set_title('Drug Concentration in Skin (DCS)')
axes[1, 1].grid(True, alpha=0.3)

# Plot 5: Drug concentration in plasma
axes[2, 0].plot(sol.t, sol.y[4], 'm-', linewidth=2)
axes[2, 0].set_xlabel('Time (sec)')
axes[2, 0].set_ylabel('Concentration (ng/mL)')
axes[2, 0].set_title('Drug Concentration in Plasma (DCP)')
axes[2, 0].grid(True, alpha=0.3)

# Plot 6: Combined skin and plasma (for comparison)
axes[2, 1].plot(sol.t, sol.y[3], 'g-', linewidth=2, label='Skin (DCS)')
axes[2, 1].plot(sol.t, sol.y[4] * 1e4, 'm--', linewidth=2, label='Plasma (DCP) × 10⁴')
axes[2, 1].set_xlabel('Time (sec)')
axes[2, 1].set_ylabel('Concentration (ng/mL)')
axes[2, 1].set_title('Skin vs Plasma Concentration')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============ PRINT SUMMARY ============
print("\n===== SIMULATION SUMMARY =====")
print(f"Total drug in microneedles: {controlMN/1e3:.2f} µg")
print(f"Total drug in baseplate: {controlBP/1e6:.2f} mg")
print(f"\nAt t = {sol.t[-1]} hr:")
print(f"  Drug released from MN: {sol.y[0][-1]/1e3:.2f} µg ({100*sol.y[0][-1]/controlMN:.1f}%)")
print(f"  Drug released from BP: {sol.y[1][-1]/1e6:.4f} mg ({100*sol.y[1][-1]/controlBP:.1f}%)")
print(f"  Drug in BP pool: {sol.y[2][-1]/1e3:.2f} µg")
print(f"  Skin concentration: {sol.y[3][-1]:.2f} ng/mL")
print(f"  Plasma concentration: {sol.y[4][-1]:.4f} ng/mL")

In [ ]:
print(sol.y[0])
print(controlMN)

### Convergence Analysis

The convergence analysis will focus on the the peak in the Drug Plasma Concentration equations. 

In [ ]:
VAR_INDEX = 4
VAR_NAME = "Drug Concentrattion in Plasma ng/mL"

def stepsize_convergence_analysis(system, t_span, y0, t_eval, method='Radau',
                                   var_index=None, var_name=None):
    T = t_span[1] - t_span[0]
    dt_values = [T / n for n in [10, 50, 100, 200, 500, 1000, 2000]]
    
    # Reference solution with stiff solver and tight tolerances
    print("Computing reference solution with Radau...")
    sol_ref = solve_ivp(system, t_span, y0, method='Radau', t_eval=t_eval,
                        rtol=1e-9, atol=1e-11)
    
    if not sol_ref.success:
        print(f"ERROR: Reference solution failed: {sol_ref.message}")
        return None, None, None, None
    
    y_ref = sol_ref.y
    print(f"Reference solution computed successfully ({sol_ref.nfev} function evals)")
    
    # Get reference max value for the variable of interest
    if var_index is not None:
        ref_max = np.max(y_ref[var_index, :])
        ref_max_time = t_eval[np.argmax(y_ref[var_index, :])]
        label = var_name if var_name else f"y[{var_index}]"
        print(f"Reference max of {label}: {ref_max:.6e} at t = {ref_max_time:.2f}\n")
    
    errors = []
    max_values = []
    print("Testing step sizes...")
    for dt in dt_values:
        sol = solve_ivp(system, t_span, y0, method=method, t_eval=t_eval,
                        max_step=dt)
        
        if not sol.success:
            print(f"  max_step = {dt:.1f}: FAILED - {sol.message}")
            errors.append(np.nan)
            max_values.append(np.nan)
        else:
            if var_index is not None:
                # Error in the maximum value of the specified variable
                sol_max = np.max(sol.y[var_index, :])
                error = np.abs(sol_max - ref_max)
                max_values.append(sol_max)
                print(f"  max_step = {dt:.1f}: max({label}) = {sol_max:.6e}, error = {error:.2e}")
            else:
                # Global max error across all variables
                error = np.max(np.abs(sol.y - y_ref))
                print(f"  max_step = {dt:.1f}: error = {error:.2e}")
            errors.append(error)
    
    return dt_values, errors, max_values, ref_max if var_index is not None else None

def plot_stepsize_convergence(dt_values, errors, var_name=None):
    """Plot convergence results for step-size analysis."""
    fig, ax = plt.subplots(figsize=(8, 6))
    
    dt_arr = np.array(dt_values)
    err_arr = np.array(errors)
    
    # Filter out zero/nan errors for log plotting
    mask = (err_arr > 1e-16) & ~np.isnan(err_arr)
    
    if np.sum(mask) < 2:
        print("\nNot enough valid data points for convergence analysis.")
        return None
    
    ax.loglog(dt_arr[mask], err_arr[mask], 'bo-', lw=2, ms=8, label='Computed error')
    
    # Estimate order of convergence
    coeffs = np.polyfit(np.log(dt_arr[mask]), np.log(err_arr[mask]), 1)
    order = coeffs[0]
    fit_line = np.exp(coeffs[1]) * dt_arr[mask]**order
    ax.loglog(dt_arr[mask], fit_line, 'r--', lw=2, label=f'Fit: order ≈ {order:.2f}')
    
    ax.set_xlabel('Maximum Step Size (Δt)', fontsize=12)
    ylabel = f'Error in max({var_name})' if var_name else 'Maximum Error'
    ax.set_ylabel(ylabel, fontsize=12)
    title = f'Convergence Analysis: {var_name}' if var_name else 'Convergence Analysis'
    ax.set_title(title, fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('stepsize_convergence.png', dpi=150)
    plt.show()
    
    print(f"\nEstimated order of convergence: {order:.2f}")
    return order

# =============================================================================
# Run the analysis
# =============================================================================
if __name__ == "__main__":
    SOLVER_METHOD = 'Radau'  # Use 'Radau' or 'BDF' for stiff systems
    
    # =========================================================================
    # CONFIGURE WHICH VARIABLE TO ANALYZE
    # =========================================================================
    # Set to None for global error across all variables, or specify:
    #   VAR_INDEX: 0=DRMN, 1=DRBP_solid, 2=DBP, 3=DCS, 4=DCP
    #   VAR_NAME: display name for plots
    VAR_INDEX = 2       # e.g., focus on DBP (index 2)
    VAR_NAME = "DBP"    # name for plot labels
    # =========================================================================
    
    print("=" * 60)
    print("STEP-SIZE CONVERGENCE ANALYSIS")
    print(f"Solver: {SOLVER_METHOD}")
    if VAR_INDEX is not None:
        print(f"Analyzing: max value of {VAR_NAME} (index {VAR_INDEX})")
    else:
        print("Analyzing: global max error across all variables")
    print("=" * 60 + "\n")
    
    dt_vals, errors, max_vals, ref_max = stepsize_convergence_analysis(
        system_of_odes, t_span, y0, t_eval, 
        method=SOLVER_METHOD,
        var_index=VAR_INDEX,
        var_name=VAR_NAME
    )
    
    if dt_vals is not None:
        order = plot_stepsize_convergence(dt_vals, errors, var_name=VAR_NAME)

### Sensitivity Analysis

Starting with a one-at-a-time approach

In [ ]:
@interact(beta=widgets.FloatSlider(value=0.185, min = 0.01, max = 0.2, step = 0.01), rho=widgets.FloatSlider(value=1.15e9, min = 1e6, max = 2e10, step = 1e8), h=widgets.FloatSlider(value=0.06, min = 0.04, max = 0.4, step = 0.05), r=widgets.FloatSlider(value=0.015, min = 0.005, max = 0.04, step = 0.005), theta=widgets.FloatSlider(value=0.244, min = 0.1, max = 0.7853, step = 0.05), hbp=widgets.FloatSlider(value=0.26, min = 0.1, max = 0.5, step = 0.05), diff=widgets.FloatSlider(value=0.3, min = 0.01, max = 10.0, step = 0.1), kd=widgets.FloatSlider(value=0.007, min = 0.001, max = 0.02, step = 0.001), nMN=widgets.IntSlider(value=100, min = 50, max = 500, step = 10), cs=widgets.FloatSlider(value=1.34e6, min = 1e4, max = 1e7, step = 1e5))
def plot_DCP_vs_time(beta, rho, h, r, theta, hbp, diff, kd, nMN, cs):
    g
    # Re-run the simulation with updated parameters
    sol = solve_ivp(system_of_odes, t_span, y0, t_eval=t_eval)
    print('Done calculating')
    return print("The maximum plasma concentration is", np.max(sol.y[4]), "ng/mL at time", sol.t[np.argmax(sol.y[4])], "sec")


In [ ]:
num_sim = 1000
#make random data
beta_vals = np.random.uniform(0.01, 0.2, num_sim)
rho_vals = np.random.uniform(1e6, 2e10, num_sim)
h_vals = np.random.uniform(0.04, 0.4, num_sim)
r_vals = np.random.uniform(0.005, 0.04, num_sim)
theta_vals = np.random.uniform(0.1, 0.7853, num_sim)
hbp_vals = np.random.uniform(0.1, 0.5, num_sim)
diff_vals = np.random.uniform(0.01, 10.0, num_sim)
kd_vals = np.random.uniform(0.001, 0.02, num_sim)
nMN_vals = np.random.randint(50, 500, num_sim)
cs_vals = np.random.uniform(1e4, 1e7, num_sim)  

data = np.array([beta_vals, rho_vals, h_vals, r_vals, theta_vals, hbp_vals, diff_vals, kd_vals, nMN_vals, cs_vals])

data.shape

In [ ]:
def system_of_odes_vals(t, y, beta_vals, rho_vals, h_vals, r_vals, theta_vals, hbp_vals, diff_vals, kd_vals, nMN_vals, cs_vals):
    DRMN_time, DRBP_solid, DBP_time, DCS_time, DCP_time = y
    # DRMN_time:  cumulative drug released from MN solid (ng)
    # DRBP_solid: cumulative drug released from BP solid (ng)
    # DBP:        dissolved drug currently in BP pool (ng)
    # DCS_time:   drug concentration in skin (ng/mL)
    # DCP_time:   drug concentration in plasma (ng/mL)
    
    # Microneedle dissolution (directly into skin)
    dDRMNdt = eqn_rate_drug_release_MN(DRMN_time, theta_vals, rho_vals, h_vals, cs_vals, DCS_time, beta_vals, kd_vals)
    if DRMN_time >= controlMN:
        dDRMNdt = 0
    
    # Baseplate solid dissolution (into BP dissolved pool)
    dDRBPdt = eqn_rate_drug_release_BP(DRBP_solid, rho_vals, Bbp, DCS_time, beta_vals, kd_vals, hbp, cs_vals)
    if DRBP_solid >= controlBP:
        dDRBPdt = 0
    
    # Flux from BP dissolved pool to skin (uses DBP, not DRBP_solid)
    [dFDBPS, dDBP_dt] = eqn_rate_flux_drugBP_2skin(nMN_vals, diff_vals, hderm, hbp_vals, DBP_time, Vbp, DCS_time, r_vals,dDRBPdt)

    
    if DRBP_solid >= controlBP and DBP_time <= 0:
        dFDBPS = 0
    
    # Mass balance for BP dissolved pool: +dissolution, -flux to skin
    dDBPdt = dDRBPdt - dFDBPS
    
    # Skin concentration
    dDCSdt = eqn_rate_drug_concentration_skin(nMN_vals, Vs, gamma, dDRMNdt, dFDBPS, DCS_time)
    
    # Plasma concentration 
    dDCPdt = eqn_change_DCP(DCS_time, Vs, Vb, gamma, Clb, DCP_time)
    
    return [dDRMNdt, dDRBPdt, dDBPdt, dDCSdt, dDCPdt]



In [ ]:
results_DRMN = []      # Cumulative MN release
results_DRBP = []      # Cumulative BP release
results_DBP = []       # BP dissolved pool
results_DCS = []       # Skin concentration
results_DCP = []       # Plasma concentration
failed_indices = []

for i in range(num_sim):
    y0 = [0.0, 0.0, 0.0, 0.0, 0.0]  # [DRMN, DRBP_solid, DBP, DCS, DCP]
    t_span = (0.0, 120000.0)

    t_eval = np.linspace(0.0, 120000.0, 100)
    sol = solve_ivp(system_of_odes_vals, t_span, y0, t_eval=t_eval, args=(beta_vals[i], rho_vals[i], h_vals[i], r_vals[i], theta_vals[i], hbp_vals[i], diff_vals[i], kd_vals[i], nMN_vals[i], cs_vals[i]))
    results_DRMN.append(sol.y[0])
    results_DRBP.append(sol.y[1])
    results_DBP.append(sol.y[2])
    results_DCS.append(sol.y[3])
    results_DCP.append(sol.y[4])




In [ ]:

# For each simulation, calculate key pharmacokinetic metrics
peak_DCP = [np.max(cp) for cp in results_DCP]           # Peak plasma concentration
time_to_peak = [t_eval[np.argmax(cp)] for cp in results_DCP]  # Time to peak
AUC_DCP = [np.trapz(cp, t_eval) for cp in results_DCP]  # Area under curve (total exposure)

peak_DCS = [np.max(cs) for cs in results_DCS]           # Peak skin concentration
total_released = [dr[-1] for dr in results_DRMN]        # Total drug released from MN

peak_DBP = [np.max(dbp) for dbp in results_DBP]

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(18, 8))

inputs = [beta_vals, rho_vals, h_vals, r_vals, theta_vals, 
          hbp_vals, diff_vals, kd_vals, nMN_vals, cs_vals]
input_names = ['β', 'ρ', 'h', 'r', 'θ', 'h_bp', 'D', 'k_d', 'n_MN', 'c_s']

# Row 1: Peak plasma concentration
for i, (x, name) in enumerate(zip(inputs, input_names)):
    axes[0, i].scatter(x, peak_DCP, alpha=0.5, s=10)
    axes[0, i].set_xlabel(name)
    axes[0, i].set_ylabel('Peak DCP')

# Row 2: AUC
for i, (x, name) in enumerate(zip(inputs, input_names)):
    axes[1, i].scatter(x, AUC_DCP, alpha=0.5, s=10)
    axes[1, i].set_xlabel(name)
    axes[1, i].set_ylabel('AUC')

plt.tight_layout()
plt.show()


In [ ]:
# Combine into dataframe
df = pd.DataFrame({
    'beta': beta_vals, 'rho': rho_vals, 'h': h_vals, 'r': r_vals,
    'theta': theta_vals, 'hbp': hbp_vals, 'diff': diff_vals,
    'kd': kd_vals, 'nMN': nMN_vals, 'cs': cs_vals,
    'peak_DCP': peak_DCP, 'AUC': AUC_DCP, 'time_to_peak': time_to_peak
})

# Correlation matrix
corr = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
plt.title('Parameter-Output Correlations')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation of each input with peak plasma concentration
correlations = [np.corrcoef(x, peak_DCP)[0, 1] for x in inputs]

# Sort by absolute value
sorted_idx = np.argsort(np.abs(correlations))
sorted_names = [input_names[i] for i in sorted_idx]
sorted_corr = [correlations[i] for i in sorted_idx]

plt.figure(figsize=(8, 6))
colors = ['red' if c < 0 else 'blue' for c in sorted_corr]
plt.barh(sorted_names, sorted_corr, color=colors)
plt.xlabel('Correlation with Peak Plasma Concentration')
plt.axvline(x=0, color='black', linewidth=0.5)
plt.title('Parameter Sensitivity Ranking')
plt.tight_layout()
plt.show()

In [82]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp

In [83]:
morris_problem = {
    # There are ten variables
    'num_vars': 10,
    # These are their names
    'names': ['beta', 'rho', 'h', 'r', 'theta', 'hbp', 'diff', 'kd', 'nMN', 'cs'],
    # These are their plausible ranges over which we'll move the variables
    'bounds': [[0.01, 0.2], 
               [1e6, 2e10], 
               [0.04, 0.4], 
               [0.005, 0.04], 
               [0.1, 0.7853], 
               [0.1, 0.5],
               [0.01, 10.0],
               [0.001, 0.02],
               [50, 500],
               [1e4, 1e7]
              ],
    # I don't want to group any of these variables together
    'groups': None
}

In [84]:
morris_problem
ms.sample
num_levels = 4
trajectories = int(1e4)
sample = ms.sample(morris_problem, trajectories, num_levels=num_levels)
sample.shape

(110000, 10)

In [85]:
print(sample.T.shape)
print(sample.T)

(10, 110000)
[[1.00000000e-02 1.00000000e-02 1.00000000e-02 ... 1.00000000e-02
  1.00000000e-02 1.00000000e-02]
 [2.00000000e+10 2.00000000e+10 2.00000000e+10 ... 2.00000000e+10
  2.00000000e+10 2.00000000e+10]
 [2.80000000e-01 2.80000000e-01 2.80000000e-01 ... 1.60000000e-01
  1.60000000e-01 4.00000000e-01]
 ...
 [7.33333333e-03 7.33333333e-03 7.33333333e-03 ... 7.33333333e-03
  7.33333333e-03 7.33333333e-03]
 [2.00000000e+02 2.00000000e+02 2.00000000e+02 ... 2.00000000e+02
  2.00000000e+02 2.00000000e+02]
 [6.67000000e+06 6.67000000e+06 6.67000000e+06 ... 1.00000000e+07
  3.34000000e+06 3.34000000e+06]]


In [ ]:
def run_model(params):
    """
    Run the ODE model for a single parameter set.
    
    Parameters
    ----------
    params : array-like
        [beta, rho, h, r, theta, hbp, diff, kd, nMN, cs]
    
    Returns
    -------
    dict : scalar outputs of interest
    """
    beta, rho, h, r, theta, hbp, diff, kd, nMN, cs = params
    
    y0 = [0.0, 0.0, 0.0, 0.0, 0.0]
    t_span = (0.0, 120000.0)
    t_eval = np.linspace(0.0, 120000.0, 500)
    
    sol = solve_ivp(
        system_of_odes_vals, 
        t_span, 
        y0,
        method='Radau',
        t_eval=t_eval,
        max_step=200,
        args=(beta, rho, h, r, theta, hbp, diff, kd, nMN, cs)
    )
    
    if not sol.success:
        # Return NaN if solver fails
        return {'peak_DCP': np.nan, 'AUC': np.nan, 'time_to_peak': np.nan}
    
    DCP = sol.y[4]  # Plasma concentration
    
    return {
        'peak_DCP': np.max(DCP),
        'AUC': np.trapz(DCP, t_eval),
        'time_to_peak': t_eval[np.argmax(DCP)]
    }

# =============================================================================
# Run model for all samples
# =============================================================================
n_samples = sample.shape[0]
print(f"Running {n_samples} simulations...")

# Initialize output arrays
peak_DCP = np.zeros(n_samples)
AUC = np.zeros(n_samples)
time_to_peak = np.zeros(n_samples)

for i, params in enumerate(sample):
    if i % 1000 == 0:
        print(f"  Progress: {i}/{n_samples}")
    
    results = run_model(params)
    peak_DCP[i] = results['peak_DCP']
    AUC[i] = results['AUC']
    time_to_peak[i] = results['time_to_peak']

print("Done!")

Running 110000 simulations...
  Progress: 0/110000


/var/folders/rl/tn37np895gj4hp73lys17b200000gn/T/ipykernel_12722/1322889380.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  'AUC': np.trapz(DCP, t_eval),


  Progress: 1000/110000
  Progress: 2000/110000
  Progress: 3000/110000
  Progress: 4000/110000
  Progress: 5000/110000
  Progress: 6000/110000


In [ ]:
Si = ma.analyze(morris_problem, 
                sample, 
                output, 
                print_to_console=False, 
                num_levels=num_levels)
print("{:20s} {:>7s} {:>7s} {:>7s}".format("Name", "mu", "mu_star", "sigma"))
for name, s1, st, mean in zip(morris_problem['names'], Si['mu'], Si['mu_star'], Si['sigma']):
    print("{:20s} {:=7.2f} {:=7.2f} {:=7.2f}".format(name, s1, st, mean))

fig, (ax1, ax2) = plt.subplots(1,2)
mp.horizontal_bar_plot(ax1, Si, param_dict={})
mp.covariance_plot(ax2, Si, {})

In [ ]:
print("\n--- Sensitivity Analysis: Peak Plasma Concentration ---")
Si_peak = ma.analyze(morris_problem, sample, peak_DCP, num_levels=num_levels)
print(Si_peak)

print("\n--- Sensitivity Analysis: AUC ---")
Si_AUC = ma.analyze(morris_problem, sample, AUC, num_levels=num_levels)
print(Si_AUC)